To run this notebook you first need to run the notebook `evaluation/compute_metrics.ipynb` or obtain the data generated for such notebook in the corresponding folders
---

In [1]:
from json import dump, load
from pprint import pprint
from statistics import median, mean, stdev
import networkx as nx

## Load gene graphs

f = open(f'../gene_graphs/vertices_inv.json', 'r')
vertices_inv = load(f)
f.close()
vertices_inv = {int(k): tuple(v) for k,v in vertices_inv.items()}

n = int(open('../gene_graphs/info', 'r').read())

components = list()
for i in range(n): ## Number of gene_graphs
    components.append(dict())
    components[i]['graph'] = nx.read_edgelist(f'../gene_graphs/graphs/component_{i+1}.edgelist', delimiter=':', create_using=nx.DiGraph, nodetype=int)
    components[i]['len'] = len(components[i]['graph'])
    
    f = open(f'../gene_graphs/sources/component_{i+1}.json', 'r')
    components[i]['sources'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/targets/component_{i+1}.json', 'r')
    components[i]['targets'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/vertex_constrains/component_{i+1}.json', 'r')
    components[i]['vertex_constrains'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/transcript_paths/component_{i+1}.json', 'r')
    components[i]['transcript_paths'] = load(f)
    f.close()
    

In [2]:
## Load experiments from json files
for i, component in enumerate(components):
    if component['len'] > 2 and len(component['transcript_paths']) > 1:
        file = open(f'../safe_paths_json/component_{i+1}.json', "r")
        dd = load(file)
        component.update(dd)
        file.close()

In [3]:
def interval_length(interval):
    return interval[1]-interval[0]+1

## It computes the base length of a transcript
def base_length(contig, vertices_inv):
    length = 0
    for v in contig:
        length += interval_length(vertices_inv[v])
    return length

In [4]:
# def choose_60_30_10_limits_cdss(components):
#     lengths = list()
#     for i, component in enumerate(components):
#         if component['len'] > 2 and len(component['transcript_paths']) > 1:
#             for CDSs in component['cdss_coverage']:
#                 for CDS in CDSs:
#                     lengths.append(CDS['length'])
#     lengths = sorted(lengths)
#     return lengths[int(0.6*len(lengths))], lengths[int(0.9*len(lengths))], lengths[-1]

# limit_cdss_60, limit_cdss_30, limit_cdss_10 = choose_60_30_10_limits_cdss(components)
# limit_cdss_60, limit_cdss_30, limit_cdss_10
limit_cdss_60, limit_cdss_30, limit_cdss_10 = 150, 500, 27705

In [5]:
# def choose_60_30_10_limits(components, vertices_inv):
#     lengths = list()
#     for i, component in enumerate(components):
#         if component['len'] > 2 and len(component['transcript_paths']) > 1:
#             for transcript in component['transcript_paths']:
#                 lengths.append(base_length(transcript['transcript_path'], vertices_inv))
#     lengths = sorted(lengths)
#     return lengths[int(0.6*len(lengths))], lengths[int(0.9*len(lengths))], lengths[-1]

# limit_60, limit_30, limit_10 = choose_60_30_10_limits(components, vertices_inv)
# limit_60, limit_30, limit_10
limit_60, limit_30, limit_10 = 2000,5000,205012

In [6]:
# def choose_60_30_10_limits_size(components, vertices_inv):
#     sizes = list()
#     for i, component in enumerate(components):
#         if component['len'] > 2 and len(component['transcript_paths']) > 1:
#             sizes.append(component['len'])
#     sizes = sorted(sizes)
#     return sizes[int(0.6*len(sizes))], sizes[int(0.9*len(sizes))], sizes[-1]

# limit_size_60, limit_size_30, limit_size_10 = choose_60_30_10_limits_size(components, vertices_inv)
# limit_size_60, limit_size_30, limit_size_10
limit_size_60, limit_size_30, limit_size_10 = 15,50,725

In [7]:
def compute_abs_improvement_table(over_width, limit_60, limit_30, components, vertices_inv):
    unitigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)

            safe_paths = component['experiments'][str(l)]['safe_paths']
            uni = component['unitigs']
            
            abs_vertex = component['experiments'][str(l)]['impr_vertex']
            abs_base = component['experiments'][str(l)]['impr_base']
            
            abs_vertex = list(map(lambda r:  r[1] - len(uni[r[0]]), enumerate(abs_vertex)))
            abs_base = list(map(lambda r:  r[1] - base_length(uni[r[0]], vertices_inv), enumerate(abs_base)))
            
                
            for j in range(len(abs_base)):
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['abs_improvements_base'] = abs_base[j]
                d['abs_improvements_vertex'] = abs_vertex[j]
                
                unitigs.append(d)

    small = list(filter(lambda c: c['component_length'] <= limit_60, unitigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , unitigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, unitigs))
    
    small_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , small))
    medium_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , medium))
    large_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , large))
    total_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , unitigs))
    
    small_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , small))
    medium_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , medium))
    large_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , large))
    total_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , unitigs))
    
    
    
    
    return { 
        'small':
        {
            'abs_improvements_vertex': 
            {
                'median': median(small_abs_improvements_vertex),
                'mean': mean(small_abs_improvements_vertex),
                'stdev': stdev(small_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(small_abs_improvements_base),
                'mean': mean(small_abs_improvements_base),
                'stdev': stdev(small_abs_improvements_base)
            },
            'number_of_unitigs': len(small)            
        },
        'medium':
        {
            'abs_improvements_vertex': 
            {
                'median': median(medium_abs_improvements_vertex),
                'mean': mean(medium_abs_improvements_vertex),
                'stdev': stdev(medium_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(medium_abs_improvements_base),
                'mean': mean(medium_abs_improvements_base),
                'stdev': stdev(medium_abs_improvements_base)
            },
            'number_of_unitigs': len(medium)
            
        },
        'large':
        {
            'abs_improvements_vertex': 
            {
                'median': median(large_abs_improvements_vertex),
                'mean': mean(large_abs_improvements_vertex),
                'stdev': stdev(large_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(large_abs_improvements_base),
                'mean': mean(large_abs_improvements_base),
                'stdev': stdev(large_abs_improvements_base)
            },
            'number_of_unitigs': len(large)
            
        },
        'total':
        {
            'abs_improvements_vertex': 
            {
                'median': median(total_abs_improvements_vertex),
                'mean': mean(total_abs_improvements_vertex),
                'stdev': stdev(total_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(total_abs_improvements_base),
                'mean': mean(total_abs_improvements_base),
                'stdev': stdev(total_abs_improvements_base)
            },
            'number_of_unitigs': len(unitigs)
            
        }
    }
    
pprint(compute_abs_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'abs_improvements_base': {'mean': 627.9201569254241,
                                     'median': 206,
                                     'stdev': 1281.652941579746},
           'abs_improvements_vertex': {'mean': 4.962040030513277,
                                       'median': 3,
                                       'stdev': 7.941987905530778},
           'number_of_unitigs': 27529},
 'medium': {'abs_improvements_base': {'mean': 775.0752131387625,
                                      'median': 227.0,
                                      'stdev': 1476.4265976426016},
            'abs_improvements_vertex': {'mean': 4.566216852345834,
                                        'median': 2.0,
                                        'stdev': 6.366215386959675},
            'number_of_unitigs': 38238},
 'small': {'abs_improvements_base': {'mean': 691.3327132086556,
                                     'median': 178,
                                     'stdev': 1387.07895

In [8]:
def compute_rel_improvement_table(over_width, limit_60, limit_30, components, vertices_inv):
    unitigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)

            uni = component['unitigs']
            safe_paths = component['experiments'][str(l)]['safe_paths']
            
            rel_vertex = component['experiments'][str(l)]['impr_vertex']
            rel_base = component['experiments'][str(l)]['impr_base']
            
            rel_vertex = list(map(lambda r:  r[1]/len(uni[r[0]]), enumerate(rel_vertex)))
            rel_base = list(map(lambda r:  r[1]/base_length(uni[r[0]], vertices_inv), enumerate(rel_base)))
            
            
                
            for j in range(len(rel_base)):
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['rel_improvements_base'] = rel_base[j]
                d['rel_improvements_vertex'] = rel_vertex[j]
                
                unitigs.append(d)

    small = list(filter(lambda c: c['component_length'] <= limit_60, unitigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , unitigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, unitigs))
    
    small_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , small))
    medium_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , medium))
    large_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , large))
    total_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , unitigs))
    
    small_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , small))
    medium_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , medium))
    large_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , large))
    total_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , unitigs))
    
    
    
    
    return { 
        'small':
        {
            'rel_improvements_vertex': 
            {
                'median': median(small_rel_improvements_vertex),
                'mean': mean(small_rel_improvements_vertex),
                'stdev': stdev(small_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(small_rel_improvements_base),
                'mean': mean(small_rel_improvements_base),
                'stdev': stdev(small_rel_improvements_base)
            },
            'number_of_unitigs': len(small)            
        },
        'medium':
        {
            'rel_improvements_vertex': 
            {
                'median': median(medium_rel_improvements_vertex),
                'mean': mean(medium_rel_improvements_vertex),
                'stdev': stdev(medium_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(medium_rel_improvements_base),
                'mean': mean(medium_rel_improvements_base),
                'stdev': stdev(medium_rel_improvements_base)
            },
            'number_of_unitigs': len(medium)
            
        },
        'large':
        {
            'rel_improvements_vertex': 
            {
                'median': median(large_rel_improvements_vertex),
                'mean': mean(large_rel_improvements_vertex),
                'stdev': stdev(large_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(large_rel_improvements_base),
                'mean': mean(large_rel_improvements_base),
                'stdev': stdev(large_rel_improvements_base)
            },
            'number_of_unitigs': len(large)
            
        },
        'total':
        {
            'rel_improvements_vertex': 
            {
                'median': median(total_rel_improvements_vertex),
                'mean': mean(total_rel_improvements_vertex),
                'stdev': stdev(total_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(total_rel_improvements_base),
                'mean': mean(total_rel_improvements_base),
                'stdev': stdev(total_rel_improvements_base)
            },
            'number_of_unitigs': len(unitigs)
            
        }
    }
    
pprint(compute_rel_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'number_of_unitigs': 27529,
           'rel_improvements_base': {'mean': 1.9285433011864033,
                                     'median': 1.1069767441860465,
                                     'stdev': 3.7314650665317286},
           'rel_improvements_vertex': {'mean': 1.5260673298684344,
                                       'median': 1.2857142857142858,
                                       'stdev': 0.895051159563057}},
 'medium': {'number_of_unitigs': 38238,
            'rel_improvements_base': {'mean': 1.7918304826432208,
                                      'median': 1.0294236475612986,
                                      'stdev': 4.164621585412872},
            'rel_improvements_vertex': {'mean': 1.4408940007010071,
                                        'median': 1.1666666666666667,
                                        'stdev': 0.7241878733178578}},
 'small': {'number_of_unitigs': 10213,
           'rel_improvements_base': {'mean': 1.410890785006288,
    

In [9]:
def compute_contigs_table(over_width, limit_60, limit_30, components, vertices_inv):
    contigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]
                
                for safe_path in safe_paths_info['safe_paths']:
                    d = dict()
                    d['component'] = i
                    d['component_length'] = component['len']
                    d['vertices'] = len(safe_path)
                    d['length'] = base_length(safe_path, vertices_inv)
                    
                    contigs.append(d)

                
                    
    
    
   
    small = list(filter(lambda c: c['component_length'] <= limit_60, contigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , contigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, contigs))
    
    small_base_lengths = list(map(lambda c: c['length'] , small))
    medium_base_lengths = list(map(lambda c: c['length'] , medium))
    large_base_lengths = list(map(lambda c: c['length'] , large))
    total_base_lengths = list(map(lambda c: c['length'] , contigs))
    
    small_vertex_lengths = list(map(lambda c: c['vertices'] , small))
    medium_vertex_lengths = list(map(lambda c: c['vertices'] , medium))
    large_vertex_lengths = list(map(lambda c: c['vertices'] , large))
    total_vertex_lengths = list(map(lambda c: c['vertices'] , contigs))
    
    
    
    
    return { 
        'small':
        {
            'vertex_lengths': 
            {
                'median': median(small_vertex_lengths),
                'mean': mean(small_vertex_lengths),
                'stdev': stdev(small_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(small_base_lengths),
                'mean': mean(small_base_lengths),
                'stdev': stdev(small_base_lengths)
            },
            'number_of_contigs': len(small)            
        },
        'medium':
        {
            'vertex_lengths': 
            {
                'median': median(medium_vertex_lengths),
                'mean': mean(medium_vertex_lengths),
                'stdev': stdev(medium_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(medium_base_lengths),
                'mean': mean(medium_base_lengths),
                'stdev': stdev(medium_base_lengths)
            },
            'number_of_contigs': len(medium)
            
        },
        'large':
        {
            'vertex_lengths': 
            {
                'median': median(large_vertex_lengths),
                'mean': mean(large_vertex_lengths),
                'stdev': stdev(large_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(large_base_lengths),
                'mean': mean(large_base_lengths),
                'stdev': stdev(large_base_lengths)
            },
            'number_of_contigs': len(large)
            
        },
        'total':
        {
            'vertex_lengths': 
            {
                'median': median(total_vertex_lengths),
                'mean': mean(total_vertex_lengths),
                'stdev': stdev(total_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(total_base_lengths),
                'mean': mean(total_base_lengths),
                'stdev': stdev(total_base_lengths)
            },
            'number_of_contigs': len(contigs)
            
        }
    }
    
pprint(compute_contigs_table(-1, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'base_lengths': {'mean': 685.3773981155637,
                            'median': 407.0,
                            'stdev': 943.095062254285},
           'number_of_contigs': 35236,
           'vertex_lengths': {'mean': 4.680752639346124,
                              'median': 4.0,
                              'stdev': 3.5487874058124858}},
 'medium': {'base_lengths': {'mean': 809.0065424967582,
                             'median': 467.0,
                             'stdev': 1110.547332118395},
            'number_of_contigs': 50898,
            'vertex_lengths': {'mean': 4.394357342135251,
                               'median': 4.0,
                               'stdev': 2.6318408446520287}},
 'small': {'base_lengths': {'mean': 1018.5952965952966,
                            'median': 582.0,
                            'stdev': 1605.5725658339256},
           'number_of_contigs': 17094,
           'vertex_lengths': {'mean': 3.365976365976366,
                     

In [10]:
def compute_contigs_table_unitigs(limit_60, limit_30, components, vertices_inv):
    contigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            unitigs = component['unitigs']

            for unitig in unitigs:
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['vertices'] = len(unitig)
                d['length'] = base_length(unitig, vertices_inv)

                contigs.append(d)
                
    
    
    small = list(filter(lambda c: c['component_length'] <= limit_60, contigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , contigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, contigs))
    
    small_base_lengths = list(map(lambda c: c['length'] , small))
    medium_base_lengths = list(map(lambda c: c['length'] , medium))
    large_base_lengths = list(map(lambda c: c['length'] , large))
    total_base_lengths = list(map(lambda c: c['length'] , contigs))
    
    small_vertex_lengths = list(map(lambda c: c['vertices'] , small))
    medium_vertex_lengths = list(map(lambda c: c['vertices'] , medium))
    large_vertex_lengths = list(map(lambda c: c['vertices'] , large))
    total_vertex_lengths = list(map(lambda c: c['vertices'] , contigs))
    
    
    
    
    return { 
        'small':
        {
            'vertex_lengths': 
            {
                'median': median(small_vertex_lengths),
                'mean': mean(small_vertex_lengths),
                'stdev': stdev(small_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(small_base_lengths),
                'mean': mean(small_base_lengths),
                'stdev': stdev(small_base_lengths)
            },
            'number_of_contigs': len(small)            
        },
        'medium':
        {
            'vertex_lengths': 
            {
                'median': median(medium_vertex_lengths),
                'mean': mean(medium_vertex_lengths),
                'stdev': stdev(medium_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(medium_base_lengths),
                'mean': mean(medium_base_lengths),
                'stdev': stdev(medium_base_lengths)
            },
            'number_of_contigs': len(medium)
            
        },
        'large':
        {
            'vertex_lengths': 
            {
                'median': median(large_vertex_lengths),
                'mean': mean(large_vertex_lengths),
                'stdev': stdev(large_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(large_base_lengths),
                'mean': mean(large_base_lengths),
                'stdev': stdev(large_base_lengths)
            },
            'number_of_contigs': len(large)
            
        },
        'total':
        {
            'vertex_lengths': 
            {
                'median': median(total_vertex_lengths),
                'mean': mean(total_vertex_lengths),
                'stdev': stdev(total_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(total_base_lengths),
                'mean': mean(total_base_lengths),
                'stdev': stdev(total_base_lengths)
            },
            'number_of_contigs': len(contigs)
            
        }
    }

pprint(compute_contigs_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'base_lengths': {'mean': 593.0802789785317,
                            'median': 324,
                            'stdev': 932.2496131797202},
           'number_of_contigs': 27529,
           'vertex_lengths': {'mean': 4.334846888735515,
                              'median': 3,
                              'stdev': 3.124333008619867}},
 'medium': {'base_lengths': {'mean': 737.7847429258852,
                             'median': 378.0,
                             'stdev': 1122.5816919163321},
            'number_of_contigs': 38238,
            'vertex_lengths': {'mean': 4.096396255034259,
                               'median': 3.0,
                               'stdev': 2.118656847870553}},
 'small': {'base_lengths': {'mean': 992.1032018016253,
                            'median': 541,
                            'stdev': 1313.9765688715343},
           'number_of_contigs': 10213,
           'vertex_lengths': {'mean': 3.4941740918437287,
                           

In [11]:
def compute_fixed_l_table(over_width, limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for j, transcript in enumerate(component['transcript_paths']):
                    transcript = transcript['transcript_path']
                    d = dict()
                    d['component'] = i
                    d['length'] = base_length(transcript, vertices_inv)
                    d['transcript'] = transcript
                    
                    d['e_size_density_vertex'] = safe_paths_info['e_sizes_vertex'][j]/len(transcript)
                    d['e_size_density_bases'] = safe_paths_info['e_size_bases'][j]/d['length']
                    d['max_prop_cov_bases'] = safe_paths_info['max_cov_bases'][j]/d['length']
                    d['max_prop_cov_vertex'] = safe_paths_info['max_cov_vertex'][j]/len(transcript)
                    
                    transcripts.append(d)
    
    
    
    small = list(filter(lambda t: t['length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['length'] > limit_30, transcripts))
    
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            }
        }
    }
    
pprint(compute_fixed_l_table(0, limit_60, limit_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.4742271574300314,
                                    'median': 0.43615924657243155,
                                    'stdev': 0.24998376718077314},
           'e_size_density_vertex': {'mean': 0.3164030582637433,
                                     'median': 0.25688705234159775,
                                     'stdev': 0.20697264953302366},
           'max_prop_cov_bases': {'mean': 0.6167484332667127,
                                  'median': 0.6108591065292096,
                                  'stdev': 0.23267466451044355},
           'max_prop_cov_vertex': {'mean': 0.4483773732626664,
                                   'median': 0.4,
                                   'stdev': 0.2165459059716942}},
 'medium': {'e_size_density_bases': {'mean': 0.45389432756518516,
                                     'median': 0.4091312869058636,
                                     'stdev': 0.24429620560156057},
            'e_size_density_ve

In [12]:
def compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            unitigs = component['unitigs']

            for j, transcript in enumerate(component['transcript_paths']):
                transcript = transcript['transcript_path']
                
                d = dict()
                d['component'] = i
                d['length'] = base_length(transcript, vertices_inv)
                d['transcript'] = transcript
                
                
                d['e_size_density_vertex'] =  component['e_sizes_vertex'][j]/len(transcript)
                d['e_size_density_bases'] =  component['e_size_bases'][j]/d['length']
                d['max_prop_cov_bases'] =  component['max_cov_bases'][j]/d['length']
                d['max_prop_cov_vertex'] =  component['max_cov_vertex'][j]/len(transcript)

                transcripts.append(d)

    
    
    small = list(filter(lambda t: t['length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['length'] > limit_30, transcripts))
    
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))

    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            }
        }
    }

pprint(compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.3729263484182285,
                                    'median': 0.3175190024986934,
                                    'stdev': 0.24396154341795023},
           'e_size_density_vertex': {'mean': 0.22167882123405824,
                                     'median': 0.17585848074921956,
                                     'stdev': 0.15666070827644976},
           'max_prop_cov_bases': {'mean': 0.5223756779628613,
                                  'median': 0.5028878305806059,
                                  'stdev': 0.24166349005332358},
           'max_prop_cov_vertex': {'mean': 0.3412612652746407,
                                   'median': 0.3,
                                   'stdev': 0.17962183650066646}},
 'medium': {'e_size_density_bases': {'mean': 0.333509782071896,
                                     'median': 0.27429914165730984,
                                     'stdev': 0.2315421466823848},
            'e_size_density_ver

In [13]:
def compute_fixed_rd_table(over_width, limit_60, limit_30, components, vertices_inv):
    genes = list()
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
                
                
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                        
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for j, transcript in enumerate(component['transcript_paths']):
                    transcript = transcript['transcript_path']
                    
                    d = dict()
                    d['component'] = i
                    d['component_length'] = component['len']
                    d['length'] = base_length(transcript, vertices_inv)
                    d['transcript'] = transcript
                    
                    
                    d['e_size_density_vertex'] = safe_paths_info['e_sizes_vertex'][j]/len(transcript)
                    d['e_size_density_bases'] = safe_paths_info['e_size_bases'][j]/d['length']
                    d['max_prop_cov_bases'] = safe_paths_info['max_cov_bases'][j]/d['length']
                    d['max_prop_cov_vertex'] = safe_paths_info['max_cov_vertex'][j]/len(transcript)
                    
                    transcripts.append(d)
                 
                d = dict()
                
                tp_bases = 0
                tp_vertex = 0
                for safe_path in safe_paths_info['true_positives']:
                    tp_bases += base_length(safe_path, vertices_inv)
                    tp_vertex += len(safe_path)
                    
                p_bases = tp_bases
                p_vertex = tp_vertex
                for safe_path in safe_paths_info['false_positives']:
                    p_bases += base_length(safe_path, vertices_inv)
                    p_vertex += len(safe_path)
                
                d['precision_bases'] = 1.0*tp_bases/p_bases
                d['precision_vertex'] = 1.0*tp_vertex/p_vertex
                d['component_length'] = components[i]['len']
                genes.append(d)
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['component_length'] > limit_30, transcripts))
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    s = list(filter(lambda t: t['component_length'] <= limit_60, genes))
    m = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , genes))
    l = list(filter(lambda t: t['component_length'] > limit_30, genes))
    
    s_precisions_bases = list(map(lambda t: t['precision_bases'] , s))
    m_precisions_bases = list(map(lambda t: t['precision_bases'] , m))
    l_precisions_bases = list(map(lambda t: t['precision_bases'] , l))
    
    s_precisions_vertex = list(map(lambda t: t['precision_vertex'] , s))
    m_precisions_vertex = list(map(lambda t: t['precision_vertex'] , m))
    l_precisions_vertex = list(map(lambda t: t['precision_vertex'] , l))
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(s_precisions_bases),
                'mean': mean(s_precisions_bases),
                'stdev': stdev(s_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(s_precisions_vertex),
                'mean': mean(s_precisions_vertex),
                'stdev': stdev(s_precisions_vertex)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(m_precisions_bases),
                'mean': mean(m_precisions_bases),
                'stdev': stdev(m_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(m_precisions_vertex),
                'mean': mean(m_precisions_vertex),
                'stdev': stdev(m_precisions_vertex)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(l_precisions_bases),
                'mean': mean(l_precisions_bases),
                'stdev': stdev(l_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(l_precisions_vertex),
                'mean': mean(l_precisions_vertex),
                'stdev': stdev(l_precisions_vertex)
            }
        }
    }
    
pprint(compute_fixed_rd_table(0, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.4065688413113546,
                                    'median': 0.3573391018253719,
                                    'stdev': 0.219150231127028},
           'e_size_density_vertex': {'mean': 0.35928651413176177,
                                     'median': 0.3125,
                                     'stdev': 0.20370434100717477},
           'max_prop_cov_bases': {'mean': 0.5527076495907087,
                                  'median': 0.5230870690731335,
                                  'stdev': 0.22431470557701236},
           'max_prop_cov_vertex': {'mean': 0.4949424169345692,
                                   'median': 0.45454545454545453,
                                   'stdev': 0.21528596221882584},
           'precision_bases': {'mean': 0.852079530823036,
                               'median': 0.9061447132215876,
                               'stdev': 0.17102711952317992},
           'precision_vertex': {'mean': 0.8352167

In [14]:
def compute_fixed_rd_table_unitigs(limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    genes = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            unitigs = component['unitigs']

            for j, transcript in enumerate(component['transcript_paths']):
                transcript = transcript['transcript_path']
                
                d = dict()
                d['component'] = i
                d['component_length'] = components[i]['len']
                d['length'] = base_length(transcript, vertices_inv)
                d['transcript'] = transcript
                
                
                d['e_size_density_vertex'] =  component['e_sizes_vertex'][j]/len(transcript)
                d['e_size_density_bases'] =  component['e_size_bases'][j]/d['length']
                d['max_prop_cov_bases'] =  component['max_cov_bases'][j]/d['length']
                d['max_prop_cov_vertex'] =  component['max_cov_vertex'][j]/len(transcript)

                transcripts.append(d)
                
            d = dict()
                
            tp_bases = 0
            tp_vertex = 0
            for safe_path in component['true_positives']:
                tp_bases += base_length(safe_path, vertices_inv)
                tp_vertex += len(safe_path)

            p_bases = tp_bases
            p_vertex = tp_vertex
            for safe_path in component['false_positives']:
                p_bases += base_length(safe_path, vertices_inv)
                p_vertex += len(safe_path)

            d['precision_bases'] = 1.0 if p_bases == 0 else 1.0*tp_bases/p_bases
            d['precision_vertex'] = 1.0 if p_vertex == 0 else 1.0*tp_vertex/p_vertex
            d['component_length'] = component['len']
            genes.append(d)

    
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['component_length'] > limit_30, transcripts))
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    s = list(filter(lambda t: t['component_length'] <= limit_60, genes))
    m = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , genes))
    l = list(filter(lambda t: t['component_length'] > limit_30, genes))
    
    s_precisions_bases = list(map(lambda t: t['precision_bases'] , s))
    m_precisions_bases = list(map(lambda t: t['precision_bases'] , m))
    l_precisions_bases = list(map(lambda t: t['precision_bases'] , l))
    
    s_precisions_vertex = list(map(lambda t: t['precision_vertex'] , s))
    m_precisions_vertex = list(map(lambda t: t['precision_vertex'] , m))
    l_precisions_vertex = list(map(lambda t: t['precision_vertex'] , l))
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(s_precisions_bases),
                'mean': mean(s_precisions_bases),
                'stdev': stdev(s_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(s_precisions_vertex),
                'mean': mean(s_precisions_vertex),
                'stdev': stdev(s_precisions_vertex)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(m_precisions_bases),
                'mean': mean(m_precisions_bases),
                'stdev': stdev(m_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(m_precisions_vertex),
                'mean': mean(m_precisions_vertex),
                'stdev': stdev(m_precisions_vertex)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(l_precisions_bases),
                'mean': mean(l_precisions_bases),
                'stdev': stdev(l_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(l_precisions_vertex),
                'mean': mean(l_precisions_vertex),
                'stdev': stdev(l_precisions_vertex)
            }
        }
    }

pprint(compute_fixed_rd_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.27227558358451914,
                                    'median': 0.2259862112793113,
                                    'stdev': 0.18737906194733775},
           'e_size_density_vertex': {'mean': 0.25209038229401776,
                                     'median': 0.21604938271604937,
                                     'stdev': 0.15654998377136004},
           'max_prop_cov_bases': {'mean': 0.4248054802601523,
                                  'median': 0.3912944453980116,
                                  'stdev': 0.20315031519003074},
           'max_prop_cov_vertex': {'mean': 0.39293081587954143,
                                   'median': 0.36363636363636365,
                                   'stdev': 0.18400718391397247},
           'precision_bases': {'mean': 1.0, 'median': 1.0, 'stdev': 0.0},
           'precision_vertex': {'mean': 1.0, 'median': 1.0, 'stdev': 0.0}},
 'medium': {'e_size_density_bases': {'mean': 0.3552156385953932

In [15]:
def compute_time_table(over_width, limit_60, limit_30, components, vertices_inv, algorithm_experiments='experiments'):
    time = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component[algorithm_experiments].get(l, None) is not None:
                safe_paths_info = component[algorithm_experiments][l]

                d = dict()
                d['time'] = component[algorithm_experiments][l]['time_main']+component[algorithm_experiments][l]['time_filter']
                d['component_length'] = component['len']
                time.append(d)
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, time))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , time))
    large = list(filter(lambda t: t['component_length'] > limit_30, time))
    
    return { 
        'small':
        {
            'time': sum(list(map(lambda x: x['time'] , small)))
        }
        ,
        'medium' :
        {
            'time': sum(list(map(lambda x: x['time'] , medium)))
        },
        'large':
        {
            'time': sum(list(map(lambda x: x['time'] , large)))
        },
        'all':
        {
            'time': sum(list(map(lambda x: x['time'] , time)))
        }
    }

pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv))
pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger'))
pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized'))
pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic'))

{'all': {'time': 11285711},
 'large': {'time': 7949952},
 'medium': {'time': 3034786},
 'small': {'time': 300973}}
{'all': {'time': 13195868},
 'large': {'time': 9769142},
 'medium': {'time': 3163780},
 'small': {'time': 262946}}
{'all': {'time': 32547900},
 'large': {'time': 25448957},
 'medium': {'time': 6714385},
 'small': {'time': 384558}}
{'all': {'time': 15927140},
 'large': {'time': 10953056},
 'medium': {'time': 4528687},
 'small': {'time': 445397}}


In [16]:
def compute_cdss_coverage_table(over_width, limit_60, limit_30, components, vertices_inv):
    cdss = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for CDSs in safe_paths_info['cdss_coverage']:
                    for CDS in CDSs:
                        d = dict()
                        d['component'] = i
                        d['length'] = CDS['length']
                        d['max_cov_rel'] = CDS['max_cov']/d['length']

                        cdss.append(d)

    
    small = list(filter(lambda t: t['length'] <= limit_60, cdss))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , cdss))
    large = list(filter(lambda t: t['length'] > limit_30, cdss))
    
    
    small_max_cov_rel = list(map(lambda t: t['max_cov_rel'] , small))
    medium_max_cov_rel = list(map(lambda t: t['max_cov_rel'] , medium))
    large_max_cov_rel = list(map(lambda t: t['max_cov_rel'] , large))

    
    
    return { 
        'small':
        {
            'max_cov_rel': 
            {
                'median': median(small_max_cov_rel),
                'mean': mean(small_max_cov_rel),
                'stdev': stdev(small_max_cov_rel)
            }
        }
        ,
        'medium' :
        {
            'max_cov_rel': 
            {
                'median': median(medium_max_cov_rel),
                'mean': mean(medium_max_cov_rel),
                'stdev': stdev(medium_max_cov_rel)
            }
        },
        'large':
        {
            'max_cov_rel': 
            {
                'median': median(large_max_cov_rel),
                'mean': mean(large_max_cov_rel),
                'stdev': stdev(large_max_cov_rel)
            }
        }
    }
    
pprint(compute_cdss_coverage_table(0, limit_cdss_60, limit_cdss_30, components, vertices_inv))

{'large': {'max_cov_rel': {'mean': 0.9894201659091227,
                           'median': 1.0,
                           'stdev': 0.062476347809396864}},
 'medium': {'max_cov_rel': {'mean': 0.9951703245960151,
                            'median': 1.0,
                            'stdev': 0.04477545037062947}},
 'small': {'max_cov_rel': {'mean': 0.9971391120287769,
                           'median': 1.0,
                           'stdev': 0.04517849777957903}}}


In [17]:
def compute_cdss_coverage_table_unitigs(limit_60, limit_30, components, vertices_inv):
    cdss = list()
    for i, component in enumerate(components):
        if component['len'] > 2 and len(component['transcript_paths']) > 1:
            for CDSs in component['cdss_coverage']:
                for CDS in CDSs:
                    d = dict()
                    d['component'] = i
                    d['length'] = CDS['length']
                    d['max_cov_rel'] = CDS['max_cov']/d['length']

                    cdss.append(d)

    
    
    small = list(filter(lambda t: t['length'] <= limit_60, cdss))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , cdss))
    large = list(filter(lambda t: t['length'] > limit_30, cdss))
    
    
    small_max_cov_rel = list(map(lambda t: t['max_cov_rel'] , small))
    medium_max_cov_rel = list(map(lambda t: t['max_cov_rel'] , medium))
    large_max_cov_rel = list(map(lambda t: t['max_cov_rel'] , large))

    
    
    return { 
        'small':
        {
            'max_cov_rel': 
            {
                'median': median(small_max_cov_rel),
                'mean': mean(small_max_cov_rel),
                'stdev': stdev(small_max_cov_rel)
            }
        }
        ,
        'medium' :
        {
            'max_cov_rel': 
            {
                'median': median(medium_max_cov_rel),
                'mean': mean(medium_max_cov_rel),
                'stdev': stdev(medium_max_cov_rel)
            }
        },
        'large':
        {
            'max_cov_rel': 
            {
                'median': median(large_max_cov_rel),
                'mean': mean(large_max_cov_rel),
                'stdev': stdev(large_max_cov_rel)
            }
        }
    }

pprint(compute_cdss_coverage_table_unitigs(limit_cdss_60, limit_cdss_30, components, vertices_inv))

{'large': {'max_cov_rel': {'mean': 0.8835126563841117,
                           'median': 1.0,
                           'stdev': 0.2700939539710111}},
 'medium': {'max_cov_rel': {'mean': 0.9104362089887359,
                            'median': 1.0,
                            'stdev': 0.24336645081378933}},
 'small': {'max_cov_rel': {'mean': 0.8984399671525961,
                           'median': 1.0,
                           'stdev': 0.2798144292933762}}}


In [18]:
## And peak memory
max(list(map(lambda x: 0 if x.get('experiments', None) is None else max(list(map(lambda k: x['experiments'][k]['peak_memory'], x['experiments'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_two_finger', None) is None else max(list(map(lambda k: x['experiments_two_finger'][k]['peak_mem'], x['experiments_two_finger'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_unoptimized', None) is None else max(list(map(lambda k: x['experiments_unoptimized'][k]['peak_mem'], x['experiments_unoptimized'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_heuristic', None) is None else max(list(map(lambda k: x['experiments_heuristic'][k]['peak_mem'], x['experiments_heuristic'].keys()))),
             components)))

(5256, 5004, 4900, 5564)

In [19]:
## LATEX TABLE GENERATORS

In [20]:
import math
def trunc(number, decimals):
    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [21]:
def merge(source, destination):
    """
    run me with nosetests --with-doctest file.py

    >>> a = { 'first' : { 'all_rows' : { 'pass' : 'dog', 'number' : '1' } } }
    >>> b = { 'first' : { 'all_rows' : { 'fail' : 'cat', 'number' : '5' } } }
    >>> merge(b, a) == { 'first' : { 'all_rows' : { 'pass' : 'dog', 'fail' : 'cat', 'number' : '5' } } }
    True
    """
    for key, value in source.items():
        if isinstance(value, dict):
            # get node or create one
            node = destination.setdefault(key, {})
            merge(value, node)
        else:
            destination[key] = value

    return destination

In [22]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{n:.2f}"

def format_cell(data, key):
    
    mdata = data[key]
    return f"""\\cellx{{2}}{{{format_number(mdata['abs_improvements_base']['mean'])}\\\\{format_number(mdata['rel_improvements_base']['mean'])}}}  & \\cellx{{2}}{{{format_number(mdata['abs_improvements_vertex']['mean'])}\\\\{format_number(mdata['rel_improvements_vertex']['mean'])}}}"""

def format_row(data, key):
    
    mdata = data[key]
    return f"""${key}$  & {format_cell(mdata, 'small')} & {format_cell(mdata, 'medium')}  & {format_cell(mdata, 'large')} \\\\\\hline"""

def format_first_paper_table(data):
    a = f"""\\begin{{center}}
\\begin{{tabular}}{{|c|cc|cc|cc|}}
\\hline
 & \\multicolumn{{2}}{{c|}}{{small graphs (3-{limit_size_60} vertices)}}  & \\multicolumn{{2}}{{c|}}{{medium graphs ({limit_size_60+1}-{limit_size_30} vertices)}} & \\multicolumn{{2}}{{c|}}{{large graphs ({limit_size_30+1}-{limit_size_10} vertices)}}    \\\\
$\\ell$  & bases     & vertices      & bases     & vertices     & bases     & vertices     \\\\\\hline\\hline
{format_row(data, 'k')}
{format_row(data, 'k+1')}
{format_row(data, 't')}
{format_row(data, '2k')}
\\end{{tabular}} 
\\end{{center}}"""
    return a


d = {
    'k': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    'k+1': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    't': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    '2k': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}}
}

d2 = {
    'k': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}},
    'k+1': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}},
    't': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}},
    '2k': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}}
}
print(format_first_paper_table(merge(d,d2)))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small graphs (3-15 vertices)}  & \multicolumn{2}{c|}{medium graphs (16-50 vertices)} & \multicolumn{2}{c|}{large graphs (51-725 vertices)}    \\
$\ell$  & bases     & vertices      & bases     & vertices     & bases     & vertices     \\\hline\hline
$k$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
$k+1$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
$t$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
$2k$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
\end{tabular} 
\end{cent

In [23]:
abs_impr_dict = dict()

abs_impr_dict['k'] = compute_abs_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv)
abs_impr_dict['k+1'] = compute_abs_improvement_table(1, limit_size_60, limit_size_30, components, vertices_inv)
abs_impr_dict['t'] = compute_abs_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
abs_impr_dict['2k'] = compute_abs_improvement_table(-2, limit_size_60, limit_size_30, components,vertices_inv)

In [24]:
rel_impr_dict = dict()

rel_impr_dict['k'] = compute_rel_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv)
rel_impr_dict['k+1'] = compute_rel_improvement_table(1, limit_size_60, limit_size_30, components, vertices_inv)
rel_impr_dict['t'] = compute_rel_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
rel_impr_dict['2k'] = compute_rel_improvement_table(-2, limit_size_60, limit_size_30, components,vertices_inv)

In [25]:
impr_dict = merge(abs_impr_dict, rel_impr_dict)
print(format_first_paper_table(impr_dict))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small graphs (3-15 vertices)}  & \multicolumn{2}{c|}{medium graphs (16-50 vertices)} & \multicolumn{2}{c|}{large graphs (51-725 vertices)}    \\
$\ell$  & bases     & vertices      & bases     & vertices     & bases     & vertices     \\\hline\hline
$k$  & \cellx{2}{691.33\\2.76}  & \cellx{2}{2.34\\1.73} & \cellx{2}{775.08\\3.52}  & \cellx{2}{4.57\\2.33}  & \cellx{2}{627.92\\3.44}  & \cellx{2}{4.96\\2.42} \\\hline
$k+1$  & \cellx{2}{160.95\\1.41}  & \cellx{2}{0.58\\1.18} & \cellx{2}{245.65\\1.80}  & \cellx{2}{1.53\\1.45}  & \cellx{2}{233.60\\1.96}  & \cellx{2}{1.93\\1.55} \\\hline
$t$  & \cellx{2}{160.75\\1.41}  & \cellx{2}{0.58\\1.18} & \cellx{2}{243.26\\1.79}  & \cellx{2}{1.50\\1.44}  & \cellx{2}{226.96\\1.93}  & \cellx{2}{1.83\\1.53} \\\hline
$2k$  & \cellx{2}{160.66\\1.41}  & \cellx{2}{0.58\\1.18} & \cellx{2}{243.25\\1.79}  & \cellx{2}{1.50\\1.44}  & \cellx{2}{226.96\\1.93}  & \cellx{2}{1.83\\1.53} \\\hline


In [26]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{trunc(n,2):.2f}"

def format_cell(data, key, variant):
    
    mdata = data[key]
    return f"""\\cell{{2}}{{{format_number(mdata['e_size_density'+variant]['mean'])}\\\\{format_number(mdata['e_size_density'+variant]['median'])}}} & \\cell{{2}}{{{format_number(mdata['max_prop_cov'+variant]['mean'])}\\\\{format_number(mdata['max_prop_cov'+variant]['median'])}}}"""

def format_row(data, key, variant):
    
    mdata = data[key]
    return f"""{format_cell(mdata, 'small', variant)} & {format_cell(mdata, 'medium', variant)} & {format_cell(mdata, 'large', variant)} \\\\\\hline"""
    

def format_second_paper_table(data, variant=''):
    
    a = f"""\\begin{{center}}
\\begin{{tabular}}{{|c|cc|cc|cc|}}
\\hline
 & \\multicolumn{{2}}{{c|}}{{small (1-{limit_60} bases)}}    & \\multicolumn{{2}}{{c|}}{{medium ({limit_60+1}-{limit_30} bases)}} & \\multicolumn{{2}}{{c|}}{{large ({limit_30+1}-{limit_10} bases)}}    \\\\
$\\ell$  & \\texttt{{esr}}     & \\texttt{{mcr}}     & \\texttt{{esr}}     & \\texttt{{mcr}}    & \\texttt{{esr}}     & \\texttt{{mcr}}    \\\\\\hline\\hline
$k$  & {format_row(data, 'k', variant)}
$k+1$  & {format_row(data, 'k+1', variant)}
$t$  & {format_row(data, 't', variant)}
$2k$  & {format_row(data, '2k', variant)}\\hline
\\cell{{1}}{{$ST$-\\\\unitigs}}
   & {format_row(data, 'unitigs', variant)}
\\end{{tabular}} 
\\end{{center}}"""

    return a

d = {
    'k': {'small': {'e_size_density': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'e_size_density': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'e_size_density': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
    'k+1': {'small': {'e_size_density': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}, 'max_prop_cov': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}},'medium': {'e_size_density': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}, 'max_prop_cov': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}},'large': {'e_size_density': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}, 'max_prop_cov': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}}},
    't': {'small': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'medium': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'large': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}}},
    '2k': {'small': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'medium': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'large': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}}},
    'unitigs': {'small': {'e_size_density': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}, 'max_prop_cov': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}},'medium': {'e_size_density': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}, 'max_prop_cov': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}},'large': {'e_size_density': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}, 'max_prop_cov': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}}},
}

print(format_second_paper_table(d))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small (1-2000 bases)}    & \multicolumn{2}{c|}{medium (2001-5000 bases)} & \multicolumn{2}{c|}{large (5001-205012 bases)}    \\
$\ell$  & \texttt{esr}     & \texttt{mcr}     & \texttt{esr}     & \texttt{mcr}    & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline
$k+1$  & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} \\\hline
$t$  & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} \\\hline
$2k$  & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.70\\

In [27]:
fixed_l_dict = dict()
fixed_l_dict['unitigs'] = compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv)
fixed_l_dict['k']  = compute_fixed_l_table(0, limit_60, limit_30, components, vertices_inv)
fixed_l_dict['k+1']  = compute_fixed_l_table(1, limit_60, limit_30, components, vertices_inv)
fixed_l_dict['t']  = compute_fixed_l_table(-1, limit_60, limit_30, components, vertices_inv)
fixed_l_dict['2k']  = compute_fixed_l_table(-2, limit_60, limit_30, components, vertices_inv)

In [28]:
print(format_second_paper_table(fixed_l_dict, '_bases'))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small (1-2000 bases)}    & \multicolumn{2}{c|}{medium (2001-5000 bases)} & \multicolumn{2}{c|}{large (5001-205012 bases)}    \\
$\ell$  & \texttt{esr}     & \texttt{mcr}     & \texttt{esr}     & \texttt{mcr}    & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{2}{0.52\\0.48} & \cell{2}{0.65\\0.64} & \cell{2}{0.45\\0.40} & \cell{2}{0.59\\0.58} & \cell{2}{0.47\\0.43} & \cell{2}{0.61\\0.61} \\\hline
$k+1$  & \cell{2}{0.49\\0.45} & \cell{2}{0.63\\0.61} & \cell{2}{0.41\\0.37} & \cell{2}{0.56\\0.54} & \cell{2}{0.43\\0.39} & \cell{2}{0.58\\0.57} \\\hline
$t$  & \cell{2}{0.48\\0.45} & \cell{2}{0.63\\0.61} & \cell{2}{0.41\\0.37} & \cell{2}{0.56\\0.54} & \cell{2}{0.43\\0.39} & \cell{2}{0.58\\0.57} \\\hline
$2k$  & \cell{2}{0.48\\0.45} & \cell{2}{0.63\\0.61} & \cell{2}{0.41\\0.37} & \cell{2}{0.56\\0.54} & \cell{2}{0.43\\0.39} & \cell{2}{0.58\\0.57} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.33\\

In [29]:
## This is the fifth table
print(format_second_paper_table(fixed_l_dict, '_vertex'))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small (1-2000 bases)}    & \multicolumn{2}{c|}{medium (2001-5000 bases)} & \multicolumn{2}{c|}{large (5001-205012 bases)}    \\
$\ell$  & \texttt{esr}     & \texttt{mcr}     & \texttt{esr}     & \texttt{mcr}    & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{2}{0.47\\0.43} & \cell{2}{0.60\\0.57} & \cell{2}{0.35\\0.29} & \cell{2}{0.48\\0.43} & \cell{2}{0.31\\0.25} & \cell{2}{0.44\\0.40} \\\hline
$k+1$  & \cell{2}{0.43\\0.40} & \cell{2}{0.57\\0.54} & \cell{2}{0.30\\0.26} & \cell{2}{0.43\\0.40} & \cell{2}{0.27\\0.22} & \cell{2}{0.40\\0.36} \\\hline
$t$  & \cell{2}{0.43\\0.39} & \cell{2}{0.57\\0.53} & \cell{2}{0.30\\0.25} & \cell{2}{0.43\\0.40} & \cell{2}{0.27\\0.22} & \cell{2}{0.40\\0.36} \\\hline
$2k$  & \cell{2}{0.43\\0.39} & \cell{2}{0.57\\0.53} & \cell{2}{0.30\\0.25} & \cell{2}{0.43\\0.40} & \cell{2}{0.27\\0.22} & \cell{2}{0.40\\0.36} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.32\\

In [30]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{trunc(n,2):.2f}"

def format_cell(data, key, variant):
    
    mdata = data[key]
    return f"""\\cell{{1}}{{{format_number(mdata['precision'+variant]['mean'])}\\\\{format_number(mdata['precision'+variant]['median'])}}}  & \\cell{{1}}{{{format_number(mdata['e_size_density'+variant]['mean'])}\\\\{format_number(mdata['e_size_density'+variant]['median'])}}}  & \\cell{{1}}{{{format_number(mdata['max_prop_cov'+variant]['mean'])}\\\\{format_number(mdata['max_prop_cov'+variant]['median'])}}}"""
    
def format_row(data, key, variant):
    
    mdata = data[key]
    return f"""{format_cell(mdata, 'small', variant)}  & {format_cell(mdata, 'medium', variant)} & {format_cell(mdata, 'large', variant)}  \\\\\\hline"""
    

def format_third_paper_table(data, variant=''):
    
    a = f"""\\begin{{center}}
    \\begin{{tabular}}{{|c|ccc|ccc|ccc|}}
\\hline
 & \\multicolumn{{3}}{{c|}}{{small graphs (3-{limit_size_60} vertices)}}        & \\multicolumn{{3}}{{c|}}{{medium graphs ({limit_size_60+1}-{limit_size_30} vertices)}}     & \\multicolumn{{3}}{{c|}}{{large graphs ({limit_size_30+1}-{limit_size_10} vertices)}}          \\\\
$\\ell$  & \\texttt{{prec}}   & \\texttt{{esr}}     & \\texttt{{mcr}}    & \\texttt{{prec}}   & \\texttt{{esr}}     & \\texttt{{mcr}}    & \\texttt{{prec}}   & \\texttt{{esr}}     & \\texttt{{mcr}}    \\\\\\hline\\hline
$k$  & {format_row(data, 'k', variant)}
$k+1$  & {format_row(data, 'k+1', variant)}
$t$  & {format_row(data, 't', variant)}
$2k$  & {format_row(data, '2k', variant)}\\hline
\\cell{{1}}{{$ST$-\\\\unitigs}} 
  & {format_row(data, 'unitigs', variant)}
\\end{{tabular}} 
\\end{{center}}"""
    
    return a

d = {
    'k': {'small': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    'k+1': {'small': {'precision': {'mean': 0.34, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    't': {'small': {'precision': {'mean': 0.45, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    '2k': {'small': {'precision': {'mean': 0.46, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    'unitigs': {'small': {'precision': {'mean': 0.56, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}}
}

print(format_third_paper_table(d))


\begin{center}
    \begin{tabular}{|c|ccc|ccc|ccc|}
\hline
 & \multicolumn{3}{c|}{small graphs (3-15 vertices)}        & \multicolumn{3}{c|}{medium graphs (16-50 vertices)}     & \multicolumn{3}{c|}{large graphs (51-725 vertices)}          \\
$\ell$  & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22} & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  \\\hline
$k+1$  & \cell{1}{0.34\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22} & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  \\\hline
$t$  & \cell{1}{0.45\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{

In [31]:
fixed_rd_dict = dict()

fixed_rd_dict['k'] = compute_fixed_rd_table(0, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['k+1'] = compute_fixed_rd_table(1, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['t'] = compute_fixed_rd_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['2k'] = compute_fixed_rd_table(-2, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['unitigs'] = compute_fixed_rd_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv)

In [32]:
print(format_third_paper_table(fixed_rd_dict, '_bases'))

\begin{center}
    \begin{tabular}{|c|ccc|ccc|ccc|}
\hline
 & \multicolumn{3}{c|}{small graphs (3-15 vertices)}        & \multicolumn{3}{c|}{medium graphs (16-50 vertices)}     & \multicolumn{3}{c|}{large graphs (51-725 vertices)}          \\
$\ell$  & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{1}{0.76\\1.00}  & \cell{1}{0.73\\0.74}  & \cell{1}{0.83\\0.88}  & \cell{1}{0.80\\0.93}  & \cell{1}{0.52\\0.48}  & \cell{1}{0.66\\0.65} & \cell{1}{0.85\\0.90}  & \cell{1}{0.40\\0.35}  & \cell{1}{0.55\\0.52}  \\\hline
$k+1$  & \cell{1}{1.00\\1.00}  & \cell{1}{0.68\\0.68}  & \cell{1}{0.81\\0.84}  & \cell{1}{0.99\\1.00}  & \cell{1}{0.48\\0.45}  & \cell{1}{0.63\\0.61} & \cell{1}{0.99\\1.00}  & \cell{1}{0.37\\0.32}  & \cell{1}{0.52\\0.49}  \\\hline
$t$  & \cell{1}{1.00\\1.00}  & \cell{1}{0.68\\0.68}  & \cell{1}{0.81\\0.84}  & \cell{1}{1.00\\1.00}  & \cell{

In [33]:
## This is the sixth table
print(format_third_paper_table(fixed_rd_dict, '_vertex'))

\begin{center}
    \begin{tabular}{|c|ccc|ccc|ccc|}
\hline
 & \multicolumn{3}{c|}{small graphs (3-15 vertices)}        & \multicolumn{3}{c|}{medium graphs (16-50 vertices)}     & \multicolumn{3}{c|}{large graphs (51-725 vertices)}          \\
$\ell$  & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{1}{0.75\\1.00}  & \cell{1}{0.64\\0.60}  & \cell{1}{0.74\\0.75}  & \cell{1}{0.79\\0.88}  & \cell{1}{0.44\\0.40}  & \cell{1}{0.57\\0.54} & \cell{1}{0.83\\0.86}  & \cell{1}{0.35\\0.31}  & \cell{1}{0.49\\0.45}  \\\hline
$k+1$  & \cell{1}{1.00\\1.00}  & \cell{1}{0.58\\0.55}  & \cell{1}{0.70\\0.66}  & \cell{1}{0.99\\1.00}  & \cell{1}{0.40\\0.36}  & \cell{1}{0.54\\0.50} & \cell{1}{0.99\\1.00}  & \cell{1}{0.32\\0.28}  & \cell{1}{0.46\\0.42}  \\\hline
$t$  & \cell{1}{1.00\\1.00}  & \cell{1}{0.58\\0.55}  & \cell{1}{0.70\\0.66}  & \cell{1}{1.00\\1.00}  & \cell{

In [34]:
def format_time(time):
    return f'{time/1000000:.2f}'

def format_fourth_table(data, rl):
    a = f"""\\begin{{table}}
\\begin{{tabular}}{{l|l|l|l|l|}}
\\cline{{2-5}}
                       & \\multicolumn{{4}}{{c|}}{{$\\ell = {rl}$}} \\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{Gene graph sets}} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{small, 3-{limit_size_60} vertices}} & {format_time(data[rl]['unoptimized']['small']['time'])}  & {format_time(data[rl]['two_finger']['small']['time'])} & {format_time(data[rl]['optimized']['small']['time'])} & {format_time(data[rl]['heuristic']['small']['time'])}\\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{medium, {limit_size_60+1}-{limit_size_30} vertices}} & {format_time(data[rl]['unoptimized']['medium']['time'])} & {format_time(data[rl]['two_finger']['medium']['time'])} & {format_time(data[rl]['optimized']['medium']['time'])} & {format_time(data[rl]['heuristic']['medium']['time'])}\\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{large, {limit_size_30+1}-{limit_size_10} vertices}} & {format_time(data[rl]['unoptimized']['large']['time'])} & {format_time(data[rl]['two_finger']['large']['time'])} & {format_time(data[rl]['optimized']['large']['time'])} & {format_time(data[rl]['heuristic']['large']['time'])}\\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{all, 3-{limit_size_10} vertices}}  & {format_time(data[rl]['unoptimized']['all']['time'])} & {format_time(data[rl]['two_finger']['all']['time'])} & {format_time(data[rl]['optimized']['all']['time'])} & {format_time(data[rl]['heuristic']['all']['time'])}\\\\ \\hline
\\end{{tabular}}
\\end{{table}}"""
    return a

d = {
    'k': {
            'optimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'heuristic': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'unoptimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'two_finger': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            }
    },
    'k+1': {
            'optimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'heuristic': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'unoptimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'two_finger': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            }
    }
}
print(format_fourth_table(d, 'k'))

\begin{table}
\begin{tabular}{l|l|l|l|l|}
\cline{2-5}
                       & \multicolumn{4}{c|}{$\ell = k$} \\ \hline
\multicolumn{1}{|l|}{Gene graph sets} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\ \hline
\multicolumn{1}{|l|}{small, 3-15 vertices} & 0.59  & 0.59 & 0.59 & 0.59\\ \hline
\multicolumn{1}{|l|}{medium, 16-50 vertices} & 0.92 & 0.92 & 0.92 & 0.92\\ \hline
\multicolumn{1}{|l|}{large, 51-725 vertices} & 23.27 & 23.27 & 23.27 & 23.27\\ \hline
\multicolumn{1}{|l|}{all, 3-725 vertices}  & 24.78 & 24.78 & 24.78 & 24.78\\ \hline
\end{tabular}
\end{table}


In [35]:
time_dict = dict()
time_dict['k'] = dict()
time_dict['k+1'] = dict()
time_dict['t'] = dict()
time_dict['2k'] = dict()

time_dict['k']['optimized'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['k']['two_finger'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['k']['unoptimized'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['k']['heuristic'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


time_dict['k+1']['optimized'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['k+1']['two_finger'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['k+1']['unoptimized'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['k+1']['heuristic'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


time_dict['t']['optimized'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['t']['two_finger'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['t']['unoptimized'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['t']['heuristic'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


time_dict['2k']['optimized'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['2k']['two_finger'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['2k']['unoptimized'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['2k']['heuristic'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


print(format_fourth_table(time_dict, 'k'))
print(format_fourth_table(time_dict, 'k+1'))
print(format_fourth_table(time_dict, 't'))
print(format_fourth_table(time_dict, '2k'))

\begin{table}
\begin{tabular}{l|l|l|l|l|}
\cline{2-5}
                       & \multicolumn{4}{c|}{$\ell = k$} \\ \hline
\multicolumn{1}{|l|}{Gene graph sets} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\ \hline
\multicolumn{1}{|l|}{small, 3-15 vertices} & 0.59  & 0.38 & 0.42 & 0.63\\ \hline
\multicolumn{1}{|l|}{medium, 16-50 vertices} & 8.41 & 3.52 & 3.38 & 5.01\\ \hline
\multicolumn{1}{|l|}{large, 51-725 vertices} & 27.65 & 9.84 & 8.05 & 11.06\\ \hline
\multicolumn{1}{|l|}{all, 3-725 vertices}  & 36.66 & 13.74 & 11.85 & 16.70\\ \hline
\end{tabular}
\end{table}
\begin{table}
\begin{tabular}{l|l|l|l|l|}
\cline{2-5}
                       & \multicolumn{4}{c|}{$\ell = k+1$} \\ \hline
\multicolumn{1}{|l|}{Gene graph sets} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\ \hline
\multicolumn{1}{|l|}{small, 3-15 vertices} & 0.38  & 0.26 & 0.30 & 0.45\\ \hline
\multicolumn{1}{|l|}{medium, 16-50 vertices} & 6.71 & 3.16 & 3.0

In [36]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{trunc(n,2):.2f}"

def format_cell(data, key, variant):
    
    mdata = data[key]
    return f"""\\cell{{2}}{{{format_number(mdata['max_cov_rel'+variant]['mean'])}\\\\{format_number(mdata['max_cov_rel'+variant]['median'])}}}"""

def format_row(data, key, variant):
    
    mdata = data[key]
    return f"""{format_cell(mdata, 'small', variant)} & {format_cell(mdata, 'medium', variant)} & {format_cell(mdata, 'large', variant)} \\\\\\hline"""
    

def format_cdss_paper_table(data, variant=''):
    
    a = f"""\\begin{{table}}[t]
\centering
\caption{{cdss max relative coverage grouped by cdss size}}
\\begin{{tabular}}{{|c|c|c|c|}}
\\hline
$\\ell$ & small (1-{limit_cdss_60} bases)    &  medium ({limit_cdss_60+1}-{limit_cdss_30} bases) & large ({limit_cdss_30+1}-{limit_cdss_10} bases)\\\\\\hline\\hline
$k$  & {format_row(data, 'k', variant)}
$k+1$  & {format_row(data, 'k+1', variant)}
$t$  & {format_row(data, 't', variant)}
$2k$  & {format_row(data, '2k', variant)}\\hline
\\cell{{1}}{{$ST$-\\\\unitigs}}
   & {format_row(data, 'unitigs', variant)}
\\end{{tabular}} 
\\end{{table}}"""

    return a

d = {
    'k': {'small': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
    'k+1': {'small': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
    't': {'small': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
    '2k': {'small': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
    'unitigs': {'small': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'max_cov_rel': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
}

print(format_cdss_paper_table(d))

\begin{table}[t]
\centering
\caption{cdss max relative coverage grouped by cdss size}
\begin{tabular}{|c|c|c|c|}
\hline
$\ell$ & small (1-150 bases)    &  medium (151-500 bases) & large (501-27705 bases)\\\hline\hline
$k$  & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline
$k+1$  & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline
$t$  & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline
$2k$  & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline
\end{tabular} 
\end{table}


In [37]:
cdss_dict = dict()
cdss_dict['unitigs'] = compute_cdss_coverage_table_unitigs(limit_cdss_60, limit_cdss_30, components, vertices_inv)
cdss_dict['k']  = compute_cdss_coverage_table(0, limit_cdss_60, limit_cdss_30, components, vertices_inv)
cdss_dict['k+1']  = compute_cdss_coverage_table(1, limit_cdss_60, limit_cdss_30, components, vertices_inv)
cdss_dict['t']  = compute_cdss_coverage_table(-1, limit_cdss_60, limit_cdss_30, components, vertices_inv)
cdss_dict['2k']  = compute_cdss_coverage_table(-2, limit_cdss_60, limit_cdss_30, components, vertices_inv)

In [38]:
print(format_cdss_paper_table(cdss_dict))

\begin{table}[t]
\centering
\caption{cdss max relative coverage grouped by cdss size}
\begin{tabular}{|c|c|c|c|}
\hline
$\ell$ & small (1-150 bases)    &  medium (151-500 bases) & large (501-27705 bases)\\\hline\hline
$k$  & \cell{2}{0.99\\1.00} & \cell{2}{0.99\\1.00} & \cell{2}{0.98\\1.00} \\\hline
$k+1$  & \cell{2}{0.99\\1.00} & \cell{2}{0.98\\1.00} & \cell{2}{0.97\\1.00} \\\hline
$t$  & \cell{2}{0.99\\1.00} & \cell{2}{0.98\\1.00} & \cell{2}{0.97\\1.00} \\\hline
$2k$  & \cell{2}{0.99\\1.00} & \cell{2}{0.98\\1.00} & \cell{2}{0.97\\1.00} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.89\\1.00} & \cell{2}{0.91\\1.00} & \cell{2}{0.88\\1.00} \\\hline
\end{tabular} 
\end{table}


In [39]:
summary_dict = {
    'impr_dict': impr_dict,
    'fixed_l_dict': fixed_l_dict,
    'fixed_rd_dict': fixed_rd_dict,
    'time_dict': time_dict,
    'cdss_dict': cdss_dict
}

f = open('./summary.json', 'w')
dump(summary_dict, f)
f.close()